# The University of Melbourne, School of Computing and Information Systems
# COMP30027 Machine Learning, 2018 Semester 1
-----
## Project 1: What is labelled data worth to Naive Bayes?
-----
###### Student Name(s): Emmanuel Macario
###### Python version: 3.6.0

This iPython notebook is a template which you may use for your Project 1 submission. (You are not required to use it; in particular, there is no need to use iPython if you do not like it.)

Marking will be applied on the seven functions that are defined in this notebook, and to your responses to the questions at the end of this notebook.

You may change the prototypes of these functions, and you may write other functions, according to your requirements. We would appreciate it if the required functions were prominent/easy to find. 

In [47]:
# Import useful libraries
from collections import defaultdict
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [48]:
# Filename constants used for easy file access
CSV1 = 'breast-cancer-dos.csv'
CSV2 = 'car-dos.csv'
CSV3 = 'hypothyroid-dos.csv'
CSV4 = 'mushroom-dos.csv'


def preprocess(filename):
    """
    Opens a data file in csv, and transforms it into
    a usable format.
    """
    
    # Read csv data into a dataframe. Assign
    # an integer value to each attribute in
    # the data.
    df = pd.read_csv(filename, header=None)
    
    
    
    # Partition the data into instances and
    # instance class labels.
    instance_list = df.iloc[:,:-1]
    class_list = df.iloc[:,-1]
    data_set = instance_list, class_list
    
    # Note: the instance list is a dataframe, whereas the class
    # list is a series object
    
    return data_set


# Test preprocess function
instance_list, class_list = preprocess(CSV2)
instance_list.head(5)

# instance_list.iloc[0,:]

,0,1,2,3,4,5
0,vhigh,vhigh,2,2,small,low
1,vhigh,vhigh,2,2,small,med
2,vhigh,vhigh,2,2,small,high
3,vhigh,vhigh,2,2,med,low
4,vhigh,vhigh,2,2,med,med


In [49]:

def train_supervised(instance_list, class_list):
    """
    Builds a supervised Naive Bays model,
    given a preprocessed set of training
    data, by calculating counts from the
    training data.
    
    Inputs: a training set of data, consisting
    of a list of instances, and a list of class 
    labels for those instances.
    
    Outputs: 2-tuple containing a class frequency 
    dictionary and also the supervised model.
    """
        
    # The supervised Naive Bays model
    supervised_model = {}
    
    # For every instance in the training
    # data, update the model
    for i in range(len(instance_list)):
        
        # Get the instance
        instance = instance_list.iloc[i,:]
        
        # Get the associated class label
        class_label = class_list[i]
        
        
        # If the class label is not already in
        # the model, create a nested set of
        # dictionaries for the class.
        if class_label not in supervised_model:
            
            # New dictionary for every new class
            supervised_model[class_label] = {}
            
            # For each attribute, initialise
            # a new default dictionary.
            for attr in instance_list.columns:
                supervised_model[class_label][attr] = defaultdict(int)
                
        
        # For every attribute in the instance,
        # get its corresponding value and update
        # the model.
        for attr in instance_list.columns:
            attr_value = instance[attr]
            supervised_model[class_label][attr][attr_value] += 1
    
    
    return supervised_model



# Test function
supervised_model = train_supervised(instance_list, class_list)

# supervised_model

In [50]:

def get_class_freqs(supervised_model):
    """
    Returns a dictionary containing frequency counts
    of all the class labels in the training data.
    """
    
    class_freqs = defaultdict(int)
    
    for class_label in supervised_model:
        class_freqs[class_label] = sum(supervised_model[class_label][0].values())
        
    return class_freqs



def predict_supervised(supervised_model, instance_list):
    """
    Predicts the class labels for a set of test
    instances, based on a supervised Naive Bayes
    model.
    """
    
    # The list of predictions for each instance
    prediction_list = []
    
    # Get the class frequencies to avoid redundant calculations
    class_freqs = get_class_freqs(supervised_model)
    
    # Calculate the total number of instances
    total_instances = sum(class_freqs.values())
    
    
    # For each instance in the test set, predict 
    # its class based on the supervised model.
    for instance in instance_list.values:
        
        max_prob = 0.0
        predicted_class = ""
        
        # We obtain the probability of an instance
        # being a certain class, for each possible class.
        for class_label in class_freqs:
            
            # Firstly, get the prior probability of a class
            prob = class_freqs[class_label]/total_instances
            
            
            # Now, multiply the prior probability of the class
            # by the posterior probability of each of the instance's 
            # attribute values, given the class.
            for attr in supervised_model[class_label]:
                prob *= (supervised_model[class_label][attr][instance[attr]]/class_freqs[class_label])
                
            
            # If the probability is the highest seen
            # thus far, set the predicted class to the
            # class label.
            if prob > max_prob:
                max_prob = prob
                predicted_class = class_label
            
        
        # Add the class label with the highest
        # corresponding probability to the list
        # of predictions.
        prediction_list.append(predicted_class)
        
        
    return prediction_list


# Test the function
prediction_list = predict_supervised(supervised_model, instance_list)

In [51]:

def evaluate_supervised(prediction_list, class_list):
    """
    Evaluates a set of predictions, in a supervised
    context. Uses accuracy as the primary method of
    evaluation.
    """
    
    # Validation checking
    assert(len(prediction_list) == len(class_list))
    
    # Calculate and return the accuracy of the model
    return (prediction_list == class_list).value_counts().loc[True]/len(prediction_list)


# Test the function
evaluate_supervised(prediction_list, class_list)


0.8738425925925926

In [55]:

def train_unsupervised(instance_list, class_labels):
    """
    Builds a weak unsupervised Naive Bayes model,
    from a given set of unlabelled training data,
    and a unique set of possible class labels.
    """
    
    # The list containing random class
    # distributions for each instance.
    random_class_distributions = []
    
    
    # For each training instance, create
    # a non-uniform class distribution for
    # that instance.
    for instance in instance_list.values:
        
        random_class_distributions.append(
            np.random.dirichlet(np.ones(len(class_labels)),size=1)[0])
        
    
    # Make a new dataframe consisting of random class distributions
    # for each instance.
    class_distributions = pd.DataFrame(random_class_distributions, 
                                       columns=class_labels)   
        
    
    # Dictionary contains the (random) frequencies
    # of each class in the training set
    class_freqs = defaultdict(int) 
    
    
    # Count the frequencies
    for class_label in class_labels:
        class_freqs[class_label] = class_distributions[class_label].sum()
    
    
    # Initialise the unsupervised Naive Bays model, which
    # is just a dictionary of dictionaries of dictionaries
    unsupervised_model = {c : {a : defaultdict(float) for a in instance_list.columns} for c in class_labels}
    
    
    # Concatenate the dataframes
    # class_distributions = pd.concat([instance_list, class_distributions], axis=1)
    

    # For every instance in the training
    # data, update the unsupervised model
    for i in range(len(instance_list)):
        
        for attr in instance_list.columns:
            
            for class_label in class_labels:
                
                unsupervised_model[class_label][attr][instance_list[attr][i]] += class_distributions[class_label][i]
                
    
    return class_distributions, class_freqs, unsupervised_model



# Test the function
class_labels = class_list.unique()
cd, cf, um = train_unsupervised(instance_list, class_labels)


In [57]:

def predict_unsupervised(class_distributions, unsupervised_model, instance_list, iterations=10):
    """
    Predicts the class distribution for a set of
    instances, based on a trained model.
    """
    
    # Get the class frequencies to avoid recalculation
    class_freqs = get_class_freqs(unsupervised_model)
    
    
    # Calculate the total number of instances
    total_instances = int(sum(class_freqs.values()) + 0.5)
    
    
    # For each instance in the test set, iteratively
    # update the class distribution for the instance
    for iteration in range(iterations):
    
        for i in range(len(instance_list)):

            # Get the instance
            instance = instance_list.iloc[i,:]


            # Find a new class distribution for the instance,
            # then normalise that distribution.
            for class_label in class_freqs:

                # Firstly, get the prior probability of a class
                prob = class_freqs[class_label]/total_instances

                # Now, multiply the prior probability of the class
                # by the posterior probability of each of the instance's 
                # attribute values, given the class.
                for attr in unsupervised_model[class_label]:
                    prob *= (unsupervised_model[class_label][attr][instance[attr]]/class_freqs[class_label])


                # Update the probability distribution for the instance
                class_distributions[class_label][i] = prob


        # Now, normalise the class distribution so that the probabilities add to 1
        class_distributions = class_distributions.div(class_distributions.sum(axis=1), axis=0)


    
    return class_distributions


predict_unsupervised(cd, um, instance_list)

,unacc,acc,vgood,good
0,0.240607,0.275175,0.239900,0.244318
1,0.233886,0.281088,0.218848,0.266177
2,0.276751,0.260007,0.237114,0.226128
3,0.247589,0.297136,0.214954,0.240321
4,0.240167,0.302883,0.195679,0.261272
5,0.284661,0.280638,0.212367,0.222334
6,0.239144,0.276237,0.239652,0.244968
7,0.232430,0.282132,0.218590,0.266847
8,0.275157,0.261095,0.236945,0.226803
9,0.240639,0.270011,0.240000,0.249350


In [9]:

def evaluate_unsupervised():
    """
    Evaluates a set of predictions, in an
    unsupervised manner.
    """
    return

Questions (you may respond in a cell or cells below):

1. Since we’re starting off with random guesses, it might be surprising that the unsupervised NB works at all. Explain what characteristics of the data cause it to work pretty well (say, within 10% Accuracy of the supervised NB) most of the time; also, explain why it utterly fails sometimes.
2. When evaluating supervised NB across the four different datasets, you will observe some variation in effectiveness (e.g. Accuracy). Explain what causes this variation. Describe and explain any particularly suprising results.
3. Evaluating the model on the same data that we use to train the model is considered to be a major mistake in Machine Learning. Implement a hold–out (hint: check out numpy.shuffle()) or cross–validation evaluation strategy. How does your estimate of Accuracy change, compared to testing on the training data? Explain why. (The result might surprise you!)
4. Implement one of the advanced smoothing regimes (add-k, Good-Turing). Do you notice any variation in the predictions made by either the supervised or unsupervised NB classifiers? Explain why, or why not.
5. The lecture suggests that deterministically labelling the instances in the initialisation phase of the unsupervised NB classifier “doesn’t work very well”. Confirm this for yourself, and then demonstrate why.
6. Rather than evaluating the unsupervised NB classifier by assigning a class deterministically, instead calculate how far away the probabilistic estimate of the true class is from 1 (where we would be certain of the correct class), and take the average over the instances. Does this performance estimate change, as we alter the number of iterations in the method? Explain why.
7. Explore what causes the unsupervised NB classifier to converge: what proportion of instances change their prediction from the random assignment, to the first iteration? From the first to the second? What is the latest iteration where you observe a prediction change? Make some conjecture(s) as to what is occurring here.

Don't forget that groups of 1 student should respond to question (1), and one other question. Groups of 2 students should respond to question (1), and three other questions. Your responses should be about 100-200 words each.